In [1]:

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from pylab import rcParams

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA # Principal Component Analysis module
from sklearn.preprocessing import StandardScaler

import ewtpy
from vmdpy import VMD
from PyLMD import LMD
import pywt

from scipy import fftpack
from scipy import signal
from scipy.linalg import eig
from scipy.signal import hilbert


import emd
from pyhht.visualization import plot_imfs
from pyhht import EMD

from PyEMD import EEMD
import pylab as plt
from PyEMD import EMD, Visualisation

import emd
from pyhht.visualization import plot_imfs
from pyhht import EMD

In [2]:
def mat_pen(y,thresh,delta_t):
        import numpy as np
    
       # matrix pencil method
       # provides 
        im_mode = y.shape[0]
        lower_V = np.floor(im_mode / 2).astype(int)
        # Initializes an empty mat_hankel matrix.
        mat_hankel = np.array(())
        # Creates a temporary mat_hankel matrix for every signal.
        for col in y:
            small_hank = np.zeros((im_mode - lower_V, lower_V + 1))
            # Iterates through every row of the mat_hankel matrix.
            for j in range(im_mode - lower_V):
                small_hank[j] = y[col][j:j + lower_V + 1]
            # Vertically stacks the mat_hankel matrices, to increase its' size with more signals.
            # The if statement is there to check for the first instance of the mat_hankel matrix, where the size is 0.
            mat_hankel = np.vstack([mat_hankel, small_hank]) if mat_hankel.size else small_hank


        # Takes a SVD of the mat_hankel matrix..
        u, sigma, vh = np.linalg.svd(mat_hankel)
        # Converts vH into v.
        v = vh.conj().transpose()
        # Gets the largest singular value.
        large_val = sigma[0]
        # Scales the values by the large singular value.
        sigma = sigma / large_val
        # Only keeps entries where the scaled value is greater than the threshold.
        sigma = sigma[sigma > thresh]
        # Rescales the singular values back to their original value.
        sigma = sigma * large_val
        # Creates a diagonal matrix with the singular values we kept.
        diag_val = np.diag(sigma)

        # Scales the size of the v matrix from SVD based on how many singular values we kept.
        num_cols = diag_val.shape[1]
        val_nu = v[:, 0:num_cols]

        # Gets the v1 and v2 matrices.
        m1 = val_nu[:-1, :]
        m2 = val_nu[1:, :]

        # Constructs the y1 and y2 matrices from v1/v2
        Y1 = np.dot(m1.transpose(), m1)
        Y2 = np.dot(m2.transpose(), m1)

        # Gets the eigenvalues of the matrix pair.
        z = sp.linalg.eig(Y2, Y1)[0]
        
        
        ct_lambda = np.log(z) / delta_t
        # Removes all complex conjugates with negative imaginary components.
        ct_lambda = np.extract(np.imag(ct_lambda) >= 0, ct_lambda)
        # Gets the list of frequencies.
        freqs = np.imag(ct_lambda) / (2 * np.pi)
        # Gets the list of damping percentages.
        damping = -100 * (np.real(ct_lambda)) / (np.sqrt(np.square(np.real(ct_lambda)) + np.square(np.imag(ct_lambda))))
        
        exp = np.arange(y.shape[0])
        exp_col = np.asmatrix(exp).transpose()
        # Constructs the Z matrix.
        z_mat = np.power(np.asmatrix(z), exp_col)
        # Solves for the mode shapes.
        shape = np.linalg.lstsq(z_mat, y, rcond=None)[0]
        # Gets the associated magnitudes and phases.
        magnitude = np.absolute(shape)
        angle = np.angle(shape)

        return(freqs,damping,magnitude,angle)

In [3]:
def instants(ewt,pc,time):
    yt = pd.DataFrame(ewt)
    rt= yt.T
    ewts = np.array(rt)
    def instant_phase(imfs):
        """Extract analytical signal through Hilbert Transform."""
        analytic_signal = hilbert(imfs)  # Apply Hilbert transform to each row
        inst_amp = np.abs(analytic_signal)
        phase = np.unwrap(np.angle(analytic_signal))  # Compute angle between img and real
        return (phase,inst_amp)
    # Define signal
    t = time
    dt = t[10]-t[9]
    S = np.array(pc)

    imfs =  ewts

    # Extract instantaneous phases and frequencies using Hilbert transform 
    instant_phases, inst_amp = instant_phase(imfs)
    instant_freqs = np.diff(instant_phases)/(2*np.pi*dt)

    # Create a figure consisting of 3 panels which from the top are the input signal, IMFs and instantaneous frequencies
    fig, axes = plt.subplots(3, figsize=(12,12))

#     # The top panel shows the input signal
#     ax = axes[0]
#     ax.plot(t, S)
#     ax.set_ylabel("Amplitude [arb. u.]")
#     ax.set_title("Input signal")

    # The middle panel shows all IMFs
#     ax = axes[1]
#     for num, imf in enumerate(imfs):
#         ax.plot(t, imf, label='IMF %s' %(num+1))

#     # Label the figure
#     ax.legend()
#     ax.set_ylabel("Amplitude [arb. u.]")
#     ax.set_title("IMFs")

    # The bottom panel shows all instantaneous frequencies
    ax = axes[2]
    for num, instant_freq in enumerate(instant_freqs):
        ax.plot(t[:-1], instant_freq, label='IMF %s'%(num+1))

    # Label the figure
    ax.legend()
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Inst. Freq. [Hz]")
    ax.set_title("Huang-Hilbert Transform")

    plt.tight_layout()
    plt.show()
    
    return(instant_freqs,instant_phases, inst_amp)

In [4]:
def rmse(predictions, targets):

    differences = predictions - targets                        #the DIFFERENCEs.

    differences_squared = differences ** 2                     #the SQUAREs of ^

    mean_of_differences_squared = differences_squared.mean()   #the MEAN of ^

    rmse_val = np.sqrt(mean_of_differences_squared)            #ROOT of ^

    return (rmse_val)     #get the ^

In [5]:
def damping_plot(inst_freq,inst_amplitude,time):
    damping = (1/inst_freq)*np.diff(np.log(inst_amplitude))*-100 
    fig, ax = plt.subplots(figsize = (20,6))
    plt.plot(time[:-1],damping)
    plt.axhline(y = 0, c = 'r', ls = ':')
    plt.ylabel('Damping ratio', size = 20)
    plt.xlabel('Time (secs)', size = 20)
    return(damping)

In [6]:
def neg_damp(a,b,c):
    damping2 = damping_plot(instant_freqs[a][b:-c],inst_amp[a][b:-c],time[b:-c])
    pos = np.abs(damping2)
    mean_damping = np.mean(damping2-pos)
    return(mean_damping)

In [7]:
def plot(ewt,time):
    t = time
    fig, ax = plt.subplots(1,figsize = (20,8))
    ewt = ewt
    K = ewt.shape[1]
    linestyles = ['b', 'g', 'm', 'c', 'k', 'r', 'k','orange','b', 'g', 'm', 'c', 'k', 'r', 'k',
                  'orange','b', 'g', 'm', 'c', 'k', 'r', 'k','orange']
    for k in range(K):
        plt.subplot(K,1,k+1)
        plt.plot(t,ewt[:,k], linestyles[k])
        plt.title('Reconstructed mode EWT %d'%(k+1))

In [8]:

def trnsform(signal,N,Fs,time):

    detect = "locmax" #detection mode: locmax, locmaxmin, locmaxminf
    reg = 'none' #spectrum regularization - it is smoothed with an average (or gaussian) filter 
    lengthFilter = 0 #length or average or gaussian filter
    sigmaFilter = 0 #sigma of gaussian filter
    f = signal
    
    ewt,  mfb ,boundaries = ewtpy.EWT1D(f, 
                                        N = N,
                                        log = 0,
                                        detect = detect, 
                                        completion = 0, 
                                        reg = reg, 
                                        lengthFilter = lengthFilter,
                                        sigmaFilter = sigmaFilter)


    t = time
    #%% show boundaries
    ff = np.fft.fft(f)
    freq=2*np.pi*np.arange(0,len(ff))/len(ff)

    if Fs !=-1:
        freq=freq*Fs/(2*np.pi)
        boundariesPLT=boundaries*Fs/(2*np.pi)
    else:
        boundariesPLT = boundaries

    ff = abs(ff[:ff.size//2])#one-sided magnitude
    freq = freq[:freq.size//2]
    #plot original signal and decomposed modes

#     fig, ax = plt.subplots(1,figsize = (12,10))

#     plt.subplot(311)
#     plt.plot(t,f)
#     plt.title('Original signal %s'%signal)

#     plt.subplot(312)
#     plt.plot(t,ewt)
#     plt.title('EWT modes')

#     plt.subplot(313)
#     plt.plot(freq,ff)
#     for bb in boundariesPLT:
#         plt.plot([bb,bb],[0,max(ff)],'b--')
#     plt.title('Spectrum partitioning')
#     plt.xlabel('Hz')
#     plt.xlim(0.1,3)
#     plt.show()
    
    return(ewt,  mfb ,boundaries)

In [9]:
def hilbert_huang(imfs,fs, time):
    
    '''takes IMFS as input
       returns instantaneous amplitude,frequency and phse'''
    
    
    fs = fs
    from scipy.signal import hilbert
    z= hilbert(imfs) #form the analytical signal
    inst_amplitude = np.abs(z) #envelope extraction
    inst_phase = np.unwrap(np.angle(z))#inst phase
    inst_freq = np.diff(inst_phase)/(2*np.pi)*fs #inst frequency
    
    fig, ax = plt.subplots(1,figsize = (20,8))
    plt.plot(time[:-1],inst_freq[3],'b'); #overlay the extracted envelope
    plt.plot(time[:-1],inst_freq[2],'r'); #overlay the extracted envelope

    plt.title('Frequency o intrinsic mode')
    plt.xlabel('time')
    plt.ylabel('Instantaneous frequency (Hz)')

    
    
    return(inst_amplitude,inst_phase, inst_freq)

In [10]:
def pca(data_without_time,n_component):
    
    
    '''takes the data without time
       returns reduced data '''
    
    
#     X = data_without_time.values
#     pca = PCA(n_components=2)
#     x_2d = pca.fit_transform(X)
    
    
    X = data_without_time.values
#     X_std = StandardScaler().fit_transform(X) #scale data
    pca = PCA(n_components=n_component)
    x_2d = pca.fit_transform(X)
    reduced_data = pd.DataFrame(x_2d)
    return (reduced_data)

In [11]:
def fft(signal, time,xlim1,xlim2):
    s = signal
    fft = np.fft.fft(s)
    t = time
    T = t[20] - t[19]  # sampling interval 
    N = s.size

    # 1/T = frequency
    f = np.linspace(0, 1 / T, N)
    fig, ax = plt.subplots(figsize = (12,4))
    plt.ylabel("Amplitude")
    plt.xlabel("Frequency [Hz]")
    plt.bar(f[:N // 2], np.abs(fft)[:N // 2] * 1 / N, width=0.03, edgecolor = 'white')  # 1 / N is a normalization factor
#     ax.axvline(x=0.53, color = 'k', label = '3 X std', linestyle = '--')
#     ax.axvline(x=0.88, color = 'k', label = '3 X std', linestyle = '--')
    plt.xlim(xlim1,xlim2)
    # plt.ylim(0,0.2)
    plt.show()

In [12]:
from PyEMD import EEMD
import pylab as plt
from PyEMD import EMD, Visualisation

In [13]:

def emdd(pc,t):
    S = pc.values

    # Assign EEMD to `eemd` variable
    eemd = EEMD()

    # Say we want detect extrema using parabolic method
    emd = eemd.EMD
    emd.extrema_detection="parabol"

    # Execute EEMD on S
    eIMFs = eemd.eemd(S, t)
    nIMFs = eIMFs.shape[0]

    # Plot results
    plt.figure(figsize=(12,9))
    plt.subplot(nIMFs+1, 1, 1)
    plt.plot(t, S, 'r')

    for n in range(nIMFs):
        plt.subplot(nIMFs+1, 1, n+2)
        plt.plot(t, eIMFs[n], 'g')
        plt.ylabel("eIMF %i" %(n+1))
        plt.locator_params(axis='y', nbins=5)

    plt.xlabel("Time [s]")
    plt.tight_layout()
    plt.savefig('eemd_example', dpi=120)
    plt.show()
    
    return(eIMFs)


In [14]:
def instants_eimf(eIMFs,pc,time):
    def instant_phase(eIMFs):
        """Extract analytical signal through Hilbert Transform."""
        analytic_signal = hilbert(imfs)  # Apply Hilbert transform to each row
        inst_amp = np.abs(analytic_signal)
        phase = np.unwrap(np.angle(analytic_signal))  # Compute angle between img and real
        return (phase,inst_amp)
    # Define signal
    t = time
    dt = t[10]-t[9]
    S = np.array(pc)

    imfs =  eIMFs

    # Extract instantaneous phases and frequencies using Hilbert transform 
    instant_phases, inst_amp = instant_phase(imfs)
    instant_freqs = np.diff(instant_phases)/(2*np.pi*dt)

    # Create a figure consisting of 3 panels which from the top are the input signal, IMFs and instantaneous frequencies
    fig, axes = plt.subplots(3, figsize=(12,12))

    # The top panel shows the input signal
    ax = axes[0]
    ax.plot(t, S)
    ax.set_ylabel("Amplitude [arb. u.]")
    ax.set_title("Input signal")

    # The middle panel shows all IMFs
    ax = axes[1]
    for num, imf in enumerate(imfs):
        ax.plot(t, imf, label='IMF %s' %(num+1))

    # Label the figure
    ax.legend()
    ax.set_ylabel("Amplitude [arb. u.]")
    ax.set_title("IMFs")

    # The bottom panel shows all instantaneous frequencies
    ax = axes[2]
    for num, instant_freq in enumerate(instant_freqs):
        ax.plot(t[:-1], instant_freq, label='IMF %s'%(num+1))

    # Label the figure
    ax.legend()
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Inst. Freq. [Hz]")
    ax.set_title("Huang-Hilbert Transform")

    plt.tight_layout()
    plt.savefig('hht_example', dpi=120)
    plt.show()
    
    return(instant_freqs,instant_phases, inst_amp)

In [15]:
def hilbert_huang(imfs,fs, time):
    
    '''takes IMFS as input
       returns instantaneous amplitude,frequency and phse'''
    
    
    fs = 50
    from scipy.signal import hilbert
    z= hilbert(imfs) #form the analytical signal
    inst_amplitude = np.abs(z) #envelope extraction
    inst_phase = np.unwrap(np.angle(z))#inst phase
    inst_freq = np.diff(inst_phase)/(2*np.pi)*fs #inst frequency
    
    fig, ax = plt.subplots(1,figsize = (12,8))
    
#     plt.plot(time[:-1],inst_freq[3],'r'); #overlay the extracted envelope
    plt.plot(time[:-1],inst_freq[2],'k'); #overlay the extracted envelope
    plt.plot(time[:-1],inst_freq[1],'b'); #overlay the extracted envelope
    plt.title('Frequency o intrinsic mode')
    plt.xlabel('time')
    plt.ylabel('Instantaneous frequency (Hz)')

    
    
    return(inst_amplitude,inst_phase, inst_freq)

In [16]:
def mat_pal(y,thresh,delta_t): 
    
        # STEP - 1
        # CREATE HANKEL MATRIX
    

        def stak(y):
            N = len(y)
            L = int(N/2)
            T = y[:N-L]
            O = y[N-L-1:]
            hg = hankel(T,O)
            return(hg)

        ht= []
        for i in range(0,y.shape[1]):
            hu = stak(y.iloc[:,i])
            ht.append(hu)

        hank_mat = np.vstack(ht)
        
        
        #STEP - 2
        # Perform a singular value decomposition of H0 to determine the number of relevant modes n
        
        u, sigma, vh = np.linalg.svd(hank_mat)
        # Converts vH into v.
        v = vh.conj().transpose()
        # Gets the largest singular value.
        large_val = sigma[0]
        # Scales the values by the large singular value.
        sigma = sigma / large_val
        # Only keeps entries where the scaled value is greater than the threshold.
        sigma = sigma[sigma > thresh]
        # Rescales the singular values back to their original value.
        sigma = sigma * large_val
        # Creates a diagonal matrix with the singular values we keep.
        diag_val = np.diag(sigma)

        # Scales the size of the v matrix from SVD based on how many singular values we kept.
        num_cols = diag_val.shape[1]
        val_nu = v[:, 0:num_cols]

        # Gets the v1 and v2 matrices.
        m1 = val_nu[:-1, :]
        m2 = val_nu[1:, :]
        
        #STEP-3
        #FIND THE REDUCED MATRIX

        # Constructs the y1 and y2 matrices from v1/v2
        Y1 = np.dot(m1.transpose(), m1)
        Y2 = np.dot(m2.transpose(), m1)
        
        #STEP4
        #FIND THE EIGEN VALUE OF REDUCED MATRICES

        # Gets the eigenvalues of the matrix pair.
        z = sp.linalg.eig(Y2, Y1)[0]
        
        
        
        #STEP5
        #FIND THE FREQUENCY AND DAMPING
        
        ct_lambda = np.log(z) / delta_t
        # Removes all complex conjugates with negative imaginary components.
        ct_lambda = np.extract(np.imag(ct_lambda) >= 0, ct_lambda)
        # Gets the list of frequencies.
        freqs = np.imag(ct_lambda) / (2 * np.pi)
        # Gets the list of damping percentages.
        damping = -100 * (np.real(ct_lambda)) / (np.sqrt(np.square(np.real(ct_lambda)) + np.square(np.imag(ct_lambda))))
        
        
        #STEP 6
        #FIND THE AMPLITUDE AND ANGLE
        
        N =y.shape[0]
        w = np.vander(z,N)
        c = w.transpose()
        c = np.flip(c, 0)
        # Solves for the mode shapes.
        B = np.linalg.lstsq(c, y, rcond=None)[0]
        # Gets the associated magnitudes and phases.
        magnitude = 2 * np.absolute(B)
        angle = np.angle(B)
        
        return(freqs,damping,magnitude,angle)